In [1]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
df = pd.read_csv('outbreaks.csv')

In [3]:
df = df[df['Food'].notnull()][df['Species'].notnull()]

/home/jackdougherty/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


In [4]:
df.head()

,Year,Month,State,Location,Food,Ingredient,Species,Serotype/Genotype,Status,Illnesses,Hospitalizations,Fatalities
3,1998,January,California,Restaurant,"Fish, Ahi",NaN,Scombroid toxin,NaN,Confirmed,4,0.0,0.0
4,1998,January,California,Private Home/Residence,"Lasagna, Unspecified; Eggs, Other",NaN,Salmonella enterica,Enteritidis,Confirmed,26,3.0,0.0
7,1998,January,California,Restaurant,"Stuffing, Unspecified; Sandwich, Turkey",NaN,Salmonella enterica,Enteritidis,Confirmed,4,3.0,0.0
8,1998,January,Colorado,Restaurant,"Salad, Unspecified",NaN,Virus,NaN,Suspected,21,NaN,NaN
9,1998,January,Colorado,Restaurant,"Peppers, Chili",NaN,Virus,NaN,Suspected,3,NaN,0.0


In [5]:
food_set = df['Food'].str.lower().str.replace(',','').str.replace(';','').str.split()
food_set.head()

3                                  [fish, ahi]
4          [lasagna, unspecified, eggs, other]
7    [stuffing, unspecified, sandwich, turkey]
8                         [salad, unspecified]
9                             [peppers, chili]
Name: Food, dtype: object

In [6]:
species_set = df['Species'].str.lower().str.replace(';',',').str.split(',').map(lambda x:[i.strip() for i in x])
species_set.head()

3        [scombroid toxin]
4    [salmonella enterica]
7    [salmonella enterica]
8                  [virus]
9                  [virus]
Name: Species, dtype: object

In [8]:
df['clean food'] = food_set

In [10]:
df['clean species'] = species_set

In [25]:
all_toxins = set()
for i in species_set:
    all_toxins |= set(i)
list(all_toxins)[0]

'campylobacter'

In [16]:
# which outbreaks were the most severe?
# which outbreaks caused the most number of people to be ill?
# which outbreaks caused the most number of hospitalizations?
# which outbreaks caused a disproportionate number of hospitalizations per illness?

In [19]:
df.sort_values('Illnesses', ascending=False).head(2)

,Year,Month,State,Location,Food,Ingredient,Species,Serotype/Genotype,Status,Illnesses,Hospitalizations,Fatalities,clean food,clean species
14169,2010,February,Multistate,Restaurant; Restaurant; Private Home/Residence...,"Shell Egg, Other",Egg,Salmonella enterica,Enteritidis,Confirmed,1939,NaN,NaN,"[shell, egg, other]",[salmonella enterica]
10396,2006,May,California,Prison/Jail,"Other Milk, Pasteurized",NaN,Campylobacter jejuni,NaN,Confirmed,1644,7.0,0.0,"[other, milk, pasteurized]",[campylobacter jejuni]


In [49]:
df['clean species'] * df['Illnesses']

3        [scombroid toxin, scombroid toxin, scombroid t...
4        [salmonella enterica, salmonella enterica, sal...
7        [salmonella enterica, salmonella enterica, sal...
8        [virus, virus, virus, virus, virus, virus, vir...
9                                    [virus, virus, virus]
                               ...                        
19105    [salmonella enterica, salmonella enterica, sal...
19108       [campylobacter unknown, campylobacter unknown]
19111    [norovirus genogroup ii, norovirus genogroup i...
19113    [clostridium perfringens, clostridium perfring...
19115    [norovirus genogroup ii, norovirus genogroup i...
Length: 7245, dtype: object

In [26]:
df1 = df

In [48]:
df1.iloc[0,:]['clean species']

['scombroid toxin']

In [78]:
toxin_severity_matrix = pd.DataFrame(columns = ['Illnesses'], index = all_toxins, data = 0)
toxin_severity_matrix.loc['scombroid toxin', 'Illnesses']

0

In [87]:
for row in df1.iterrows():
    for t in all_toxins:
        if t in row[1]['clean species']:
            toxin_severity_matrix[t] += row[1]['Illnesses']

KeyError: 'scombroid toxin'